In [1]:
from typing import List
from ultralytics import YOLO

In [2]:
citypersons = YOLO("runs/detect/train2/weights/best.pt")
deepdrive = YOLO("runs/detect/train3/weights/best.pt")
kitti = YOLO("runs/detect/train4/weights/best.pt")
roadsigns = YOLO("runs/detect/train5/weights/best.pt")

In [3]:
class YoloClient:
    def __init__(self, model: YOLO, dataset: str):
        self.model: YOLO = model
        self.dataset = dataset
        self.config_path = f"datasets/{dataset}/config.yaml"

    def train(self, epochs):
        self.model.train(data=self.config_path, epochs=epochs)
    
    def set_parameters(self, state_dict):
        self.model.load_state_dict(state_dict)

    def get_parameters(self):
        return self.model.state_dict()
    
    def __repr__(self):
        return f"YoloClient({self.dataset})"

In [4]:
models = [citypersons, deepdrive, kitti, roadsigns]
datasets = ["citypersons", "deepdrive", "kitti", "roadsigns"]
clients = [YoloClient(model, dataset) for model, dataset in zip(models, datasets)]

In [5]:
class YoloServer:
    def __init__(self, model: YOLO, clients: List[YoloClient]):
        self.model = model
        self.clients = clients

    def start(self, rounds=1, epochs=1):
        for round in range(rounds):
            print(f"Starting round {round}/{rounds}")
            for client in self.clients:
                client.set_parameters(self.model.state_dict())
                client.train(epochs)
            self.update([client.get_parameters() for client in self.clients])
    
    def update(self, state_dicts):
        avg_state_dict = {}
        for key in self.model.state_dict().keys():
            avg_state_dict[key] = sum(state_dict[key] for state_dict in state_dicts) / len(state_dicts)
        self.model.load_state_dict(avg_state_dict)

: 

In [6]:
model = YOLO("runs/detect/train3/weights/best.pt")
server = YoloServer(model, clients)
server.start()

Starting round 0/1
New https://pypi.org/project/ultralytics/8.2.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.33 🚀 Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train2/weights/best.pt, data=datasets/citypersons/config.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train19, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cl

/home/pierre/venvs/base/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /home/pierre/git/edgeAI/datasets/citypersons/labels/train.cache... 2975 images, 493 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:00<?, ?it/s]
val: Scanning /home/pierre/git/edgeAI/datasets/citypersons/labels/val.cache... 500 images, 61 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to runs/detect/train19/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000476, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train19
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1497       8005      0.565       0.34      0.388      0.243

1 epochs completed in 0.018 hours.
Optimizer stripped from runs/detect/train21/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train21/weights/best.pt, 6.2MB

Validating runs/detect/train21/weights/best.pt...
Ultralytics YOLOv8.2.33 🚀 Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
Model summary (fused): 168 layers, 3008963 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:05<00:00,  8.18it/s]


                   all       1497       8005      0.566      0.341      0.389      0.243
                   Car       1335       5667      0.708      0.854      0.871      0.601
            Pedestrian        339        948      0.592      0.582      0.609      0.309
                   Van        417        565      0.606      0.302      0.399      0.266
                 Rider        219        326      0.537      0.274      0.336      0.163
                 Truck        181        193      0.608      0.456      0.552      0.403
                  misc        149        180          0          0     0.0348      0.018
                  Tram         63         93      0.474      0.258      0.303      0.185
        Person sitting         19         33          1          0    0.00548     0.0021
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train21
New https://pypi.org/project/ultralytics/8.2.34 available 😃 Update with 'pip ins

train: Scanning /home/pierre/git/edgeAI/datasets/roadsigns/labels/train.cache... 702 images, 0 backgrounds, 0 corrupt: 100%|██████████| 702/702 [00:00<?, ?it/s]
val: Scanning /home/pierre/git/edgeAI/datasets/roadsigns/labels/val.cache... 175 images, 0 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]


Plotting labels to runs/detect/train24/labels.jpg... 
